![Podmanlogo](Pictures/podman-logo.png)

# What is Podman?

As described by the Podman website itself it is:

    Podman is a daemonless, open source, Linux native tool designed to make it easy to find, run, build, share and deploy applications using Open Containers Initiative (OCI) Containers and Container Images. Podman provides a command line interface (CLI) familiar to anyone who has used the Docker Container Engine. Most users can simply alias Docker to Podman (alias docker=podman) without any problems. Similar to other common Container Engines (Docker, CRI-O, containerd), Podman relies on an OCI compliant Container Runtime (runc, crun, runv, etc) to interface with the operating system and create the running containers. This makes the running containers created by Podman nearly indistinguishable from those created by any other common container engine.

With simple words it's a container engine that is so similar to Docker that you can just use an alias and run docker-like commands. This brings to mind an obvious question, if it's so similar to Docker why would you use Podman instead? The answer is security.
The previous paragraph starts describing Podman as a "daemonless" container engine and that's the main difference with Docker. Later in this workshop we'll explain security features in Podman, for the moment just keep in mind that as Podman doesn't have a daemon it doesn't need to use the root user for anything, therefore the execution of containers is secured by design. There are other benefits of using a daemonless architecture like better performance.

This is explained differently in Podman website:

    Containers under the control of Podman can either be run by root or by a non-privileged user. Podman manages the entire container ecosystem which includes pods, containers, container images, and container volumes using the libpod library. Podman specializes in all of the commands and functions that help you to maintain and modify OCI container images, such as pulling and tagging. It allows you to create, run, and maintain those containers and container images in a production environment.

Moreover, podman has gone further and brings some new functionalities that will help enterprises looking at container adoption for their workloads. We will explain some of these during this workshop.

# Podman hello world: pulling and executing your first container image

Just like any other container engine, Podman has the ability to manage and execute container images. A container image is a static image that contains an applications with all it's dependencies. Because the dependencies are included within the container image we can expect the same application behaviour if it's executed in different hosts, solving one of the most widespread problems for developers: "it worked in my environment".

Container images are stored in container registries, by default Podman will look for container images in quay.io, registry.redhat.io and docker.io. You can configure additional registries by modifying the file /etc/containers/registries.conf, we'll not cover this advanced configuration during this workshop.

The way Podman works with container images is it locally downloads (or pulls) images from registry. Hence, first thing we want to do is checking if we have any image downloaded locally:

In [ ]:
%login student{{ STDID }}@ {{ IP-WKSHP-Podman101 }}

In [1]:
%login student491@16.31.86.187

[ssh] Login to student491@16.31.86.187...
[ssh] host=student491@16.31.86.187 hostname=16.31.86.187 other_conf={'user': 'student491', 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f6b4c58bdc0>}
[ssh] Successfully logged in.


In [ ]:
podman images

You should have no images locally. Download your first image, the podman hello-world image, by running the following command:

In [ ]:
podman pull hello-world

Now check if your image was pulled correctly by running again the podman images command:

In [ ]:
podman images

You should find the hello-world image is locally downloaded in your system. This only means that we have a static image that contains our application and its dependancies locally sync'ed. Next thing we would like to do is to execute that static image:

In [ ]:
podman run hello-world

# Executing containerized applications

During last exercise we downloaded a container image and executed it. With the following command Podman will show the containers that are being executed right now:

In [ ]:
podman ps

As you can see, the output is empty. This is because the container that we executed runs a process that ends after printing in the screen the "Hello Podman World" message and once the main process of a container is finished the container itself will stop being executed.
But no worries, Podman got us covered, if we want to see all the containers that are being executed and those whose execution finished some time ago we just need to add the "-a" to the previous command.

In [ ]:
podman ps -a

There you can see the container we executed before with a status of "Exited".

Lets see what happens if we execute a container whose process don't finish unless specified:

In [2]:
podman run nginx

[ssh] host = student491@16.31.86.187, cwd = /student/student491
/docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
/docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
/docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
10-listen-on-ipv6-by-default.sh: info: Getting the checksum of /etc/nginx/conf.d/default.conf
10-listen-on-ipv6-by-default.sh: info: Enabled listen on IPv6 in /etc/nginx/conf.d/default.conf
/docker-entrypoint.sh: Sourcing /docker-entrypoint.d/15-local-resolvers.envsh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/30-tune-worker-processes.sh
/docker-entrypoint.sh: Configuration complete; ready for start up
2023/10/27 07:59:47 [notice] 1#1: using the "epoll" event method
2023/10/27 07:59:47 [notice] 1#1: nginx/1.25.3
2023/10/27 07:59:47 [notice] 1#1: built by gcc 12.2.0 (Debian 12.2.0-

* interrupt...
Traceback (most recent call last):
  File "/opt/jupyterhub/lib/python3.8/site-packages/sshkernel/kernel.py", line 115, in do_execute_direct
    exitcode = self.sshwrapper.exec_command(code, self.Write)
  File "/opt/jupyterhub/lib/python3.8/site-packages/sshkernel/ssh_wrapper_plumbum.py", line 51, in exec_command
    env_info = process_output(tuple_iterator, marker, print_function)
  File "/opt/jupyterhub/lib/python3.8/site-packages/sshkernel/ssh_wrapper_plumbum.py", line 204, in process_output
    for line in iterator:
  File "/opt/jupyterhub/lib/python3.8/site-packages/sshkernel/ssh_wrapper_plumbum.py", line 181, in merge_stdout_stderr
    for (stdout, stderr) in iterator:
  File "/opt/jupyterhub/lib/python3.8/site-packages/plumbum/commands/processes.py", line 367, in iter_lines
    for t, line in _iter_lines(proc, decode, linesize, line_timeout):
  File "/opt/jupyterhub/lib/python3.8/site-packages/plumbum/machines/paramiko_machine.py", line 508, in _iter_lines
    for 

abort: 1

You need to stop the process manually as the container process has taken all of your terminal. To do that you need to press the stop button on top of the jupyter notebook, as shown in this picture:

ADD IMAGE

If you want to run a container in the background, then you should add the option --detach (or simply -d).

In [3]:
podman run --detach nginx

[ssh] host = student491@16.31.86.187, cwd = /student/student491
ca93056a7608ef5df74a49a0541951a07355a2c50f42f078eb9eb48be927b0c5


Now, if you run the "podman ps" command you will see your nginx container with a status of "Up".

In [4]:
podman ps

[ssh] host = student491@16.31.86.187, cwd = /student/student491
CONTAINER ID  IMAGE                           COMMAND               CREATED        STATUS        PORTS       NAMES
cbc0023b3673  docker.io/library/nginx:latest  nginx -g daemon o...  4 minutes ago  Up 4 minutes              blissful_cannon
87832e67af0a  docker.io/library/nginx:latest  nginx -g daemon o...  3 minutes ago  Up 3 minutes              frosty_edison
741da060fc7f  docker.io/library/nginx:latest  nginx -g daemon o...  3 minutes ago  Up 3 minutes              gallant_hypatia
ca93056a7608  docker.io/library/nginx:latest  nginx -g daemon o...  4 seconds ago  Up 4 seconds              determined_dirac


You see the container has an ID and also a name. In this case the name was randomly generated as we did not specify any name, pay attention to next time we execute a container image as we will be giving it a name.

You can stop all running containers by running the following:

In [5]:
podman stop --all

[ssh] host = student491@16.31.86.187, cwd = /student/student491
6a307bb7fd53cd25b66957fa2e51080f65964be4bee6ec7763abbf6238f3dae1
741da060fc7f1d5ddcf6dc195474e846557dc035fa34632f93cea8384ccffa4c
87832e67af0a994c07e8ffc8f5c0551593051ccd78875f5dd35fb22664d06e5f
ca93056a7608ef5df74a49a0541951a07355a2c50f42f078eb9eb48be927b0c5
cbc0023b3673b298b1249528171aea596ba1c53787cb8211509211227351f5f7


Now that all containers have beeen stopped run again "podman ps" to check if all of them were correctly stopped.

In [6]:
podman ps

[ssh] host = student491@16.31.86.187, cwd = /student/student491
CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


As you can see the container is not present because "podman ps" only shows the containers that are being executed right now. If we use the "-a" option it will show all containers that are in execution right now but also those that were stopped, it'll show the stopped ones with an status of "Exited".

In [8]:
podman ps -a

[ssh] host = student491@16.31.86.187, cwd = /student/student491
CONTAINER ID  IMAGE                           COMMAND               CREATED         STATUS                    PORTS       NAMES
6a307bb7fd53  quay.io/podman/hello:latest     /usr/local/bin/po...  6 minutes ago   Exited (0) 6 minutes ago              sad_gould
cbc0023b3673  docker.io/library/nginx:latest  nginx -g daemon o...  5 minutes ago   Exited (0) 9 seconds ago              blissful_cannon
87832e67af0a  docker.io/library/nginx:latest  nginx -g daemon o...  4 minutes ago   Exited (0) 9 seconds ago              frosty_edison
741da060fc7f  docker.io/library/nginx:latest  nginx -g daemon o...  3 minutes ago   Exited (0) 9 seconds ago              gallant_hypatia
ca93056a7608  docker.io/library/nginx:latest  nginx -g daemon o...  18 seconds ago  Exited (0) 9 seconds ago              determined_dirac


This means that containers are not deleted once the process is finished and you could simply restart any of them. We can do this by using the container ID, but for the sake of simplicity we will be creating a new container with a defined name:

In [9]:
podman run -d --name my-podman-container nginx
podman ps

[ssh] host = student491@16.31.86.187, cwd = /student/student491
71864687721cfca31e315c34f00dbbafca8bcf7f1dd3d207f57778006c985d5c
CONTAINER ID  IMAGE                           COMMAND               CREATED                 STATUS                 PORTS       NAMES
71864687721c  docker.io/library/nginx:latest  nginx -g daemon o...  Less than a second ago  Up Less than a second              my-podman-container


See how we changed the long "--detach" option for a short "-d" but it works the same way. We also defined the name. So now we can just stop it by it's name.

In [10]:
podman stop my-podman-container
podman ps -a

[ssh] host = student491@16.31.86.187, cwd = /student/student491
my-podman-container
CONTAINER ID  IMAGE                           COMMAND               CREATED         STATUS                             PORTS       NAMES
6a307bb7fd53  quay.io/podman/hello:latest     /usr/local/bin/po...  6 minutes ago   Exited (0) 6 minutes ago                       sad_gould
cbc0023b3673  docker.io/library/nginx:latest  nginx -g daemon o...  5 minutes ago   Exited (0) 13 seconds ago                      blissful_cannon
87832e67af0a  docker.io/library/nginx:latest  nginx -g daemon o...  4 minutes ago   Exited (0) 13 seconds ago                      frosty_edison
741da060fc7f  docker.io/library/nginx:latest  nginx -g daemon o...  3 minutes ago   Exited (0) 13 seconds ago                      gallant_hypatia
ca93056a7608  docker.io/library/nginx:latest  nginx -g daemon o...  23 seconds ago  Exited (0) 13 seconds ago                      determined_dirac
71864687721c  docker.io/library/nginx:latest  nginx

Now that it is stopped, we can restart it.

In [11]:
podman start my-podman-container
podman ps

[ssh] host = student491@16.31.86.187, cwd = /student/student491
my-podman-container
CONTAINER ID  IMAGE                           COMMAND               CREATED        STATUS                 PORTS       NAMES
71864687721c  docker.io/library/nginx:latest  nginx -g daemon o...  4 seconds ago  Up Less than a second              my-podman-container


While the container is running you can also inspect it to get to know all the information about it.

In [12]:
podman inspect my-podman-container

[ssh] host = student491@16.31.86.187, cwd = /student/student491
[
     {
          "Id": "71864687721cfca31e315c34f00dbbafca8bcf7f1dd3d207f57778006c985d5c",
          "Created": "2023-10-27T04:03:29.058128252-04:00",
          "Path": "/docker-entrypoint.sh",
          "Args": [
               "nginx",
               "-g",
               "daemon off;"
          ],
          "State": {
               "OciVersion": "1.1.0-rc.1",
               "Status": "running",
               "Running": true,
               "Paused": false,
               "Restarting": false,
               "OOMKilled": false,
               "Dead": false,
               "Pid": 239303,
               "ConmonPid": 239292,
               "ExitCode": 0,
               "Error": "",
               "StartedAt": "2023-10-27T04:03:33.60964276-04:00",
               "FinishedAt": "2023-10-27T04:03:31.218606563-04:00",
               "Health": {
                    "Status": "",
                    "FailingStreak": 0,
         

If we stop it again we see that we can also remove it by running the command "podman rm"

In [13]:
podman stop my-podman-container
podman rm my-podman-container

[ssh] host = student491@16.31.86.187, cwd = /student/student491
my-podman-container
my-podman-container


Now it has been fully removed so we wont be able to start that same container again, we will need to create a new container from the same image. You can check it is fully removed by looking at the empty output of "podman ps -a"

In [14]:
podman ps -a

[ssh] host = student491@16.31.86.187, cwd = /student/student491
CONTAINER ID  IMAGE                           COMMAND               CREATED         STATUS                     PORTS       NAMES
6a307bb7fd53  quay.io/podman/hello:latest     /usr/local/bin/po...  6 minutes ago   Exited (0) 6 minutes ago               sad_gould
cbc0023b3673  docker.io/library/nginx:latest  nginx -g daemon o...  5 minutes ago   Exited (0) 24 seconds ago              blissful_cannon
87832e67af0a  docker.io/library/nginx:latest  nginx -g daemon o...  4 minutes ago   Exited (0) 24 seconds ago              frosty_edison
741da060fc7f  docker.io/library/nginx:latest  nginx -g daemon o...  3 minutes ago   Exited (0) 24 seconds ago              gallant_hypatia
ca93056a7608  docker.io/library/nginx:latest  nginx -g daemon o...  34 seconds ago  Exited (0) 24 seconds ago              determined_dirac


A very common question is how to modify the configuration of a running container. There are several ways you can do this, one of the most common is to run the container interactively. You will need to add two options for this purpose, first one is the "--tty" that will allocate a pseudo-TTY for the container; the second is "--interactive" which will keep STDIN open even if not attached. We can combine both options just by using the "-it" option.
Then you will need to specify the command you want to run in your container. If you specify "/bin/bash" as the command you want to run you will open an interactive terminal with your container.

In our case, this environment based on jupyter notebooks doesn't work well with interactive commands. This is why instead of using "/bin/bash" command we will use the command "sleep 300" that will start our container as a process running for the next 300 seconds.

In [15]:
podman run -it -d --name my-podman-container fedora sleep 300

[ssh] host = student491@16.31.86.187, cwd = /student/student491
time="2023-10-27T04:03:44-04:00" level=warning msg="The input device is not a TTY. The --tty and --interactive flags might not work properly"
Resolved "fedora" as an alias (/etc/containers/registries.conf.d/000-shortnames.conf)
Trying to pull registry.fedoraproject.org/fedora:latest...
Getting image source signatures
Copying blob sha256:b30887322388a8cac2870fb8c3967da520319cb02219923e87b60ca980ccb572
Copying config sha256:a1cd3cbf8adaa422629f2fcdc629fd9297138910a467b11c66e5ddb2c2753dff
Writing manifest to image destination
Storing signatures
255f4ab360f4ffa2504a47bb5008bb28310e0a959677bcb6ec10ac9ec5486f1f


If the container is already running you can just use the "podman exec" command and then specify the command that you want to run. Again, if you use "/bin/bash" you'll have access to the shell inside the container but you can just run any other command as shown in the example below.

In [17]:
podman exec -it my-podman-container cat /etc/fedora-release

[ssh] host = student491@16.31.86.187, cwd = /student/student491
Fedora release 38 (Thirty Eight)


As you can see the output showed the version of Fedora running within the container.

# Cleanup

In [21]:
podman stop --all
podman rm --all
podman rmi hello-world nginx fedora

[ssh] host = student491@16.31.86.187, cwd = /student/student491
CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


In [22]:
%logout

[ssh] Closing existing connection.
[ssh] Successfully logged out.
